In [1]:
import findspark
from pyspark.sql import SparkSession
from pyspark.storagelevel import StorageLevel

findspark.init()
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/07/18 14:15:57 WARN Utils: Your hostname, RNTDELL000700 resolves to a loopback address: 127.0.1.1; using 192.168.1.131 instead (on interface wlp0s20f3)
23/07/18 14:15:57 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/18 14:15:58 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/07/18 14:15:58 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
sc = spark.sparkContext

Cree un RDD importes a partir de los datos adjuntos a esta lección como recurso. Emplee acumuladores para obtener el total de ventas realizadas y el importe total de las ventas.

In [3]:
importes = sc.textFile('./data/rdd.txt')

In [4]:
# Comprobamos que los datos están en forma de string, no de int
data_type = importes.map(lambda x: type(x).__name__).collect()
print(set(data_type))

{'str'}


In [5]:
# Cambiamos los datos a int
importes_int = importes.map(lambda x: float(x))
importes_int.take(10)

[527.0, 386.0, 701.0, 240.0, 941.0, 27.0, 396.0, 56.0, 456.0, 148.0]

In [6]:
acumuladorVentas = sc.accumulator(0)
acumuladorImporte = sc.accumulator(0)

In [7]:
importes_int.foreach(lambda x: acumuladorVentas.add(1))
importes_int.foreach(lambda x: acumuladorImporte.add(x))

In [8]:
print('El número total de ventas realizadas es ' + str(acumuladorVentas.value))
print('El número importe total de las ventas es ' + str(acumuladorImporte.value))

El número total de ventas realizadas es 10000
El número importe total de las ventas es 5042335.0


Si se conoce que a cada venta hay que restarle un importe fijo igual a 10 pesos por temas de impuestos.

    1.-¿Cómo restaría este impuesto de cada venta utilizando una variable broadcast para acelerar el proceso?
    2.-Cree un RDD llamado ventas_sin_impuestos a partir de la propuesta del inciso 1 que contenga las ventas sin impuestos.
    3.-Destruya la variable broadcast creada luego de emplearla para crear el RDD del inciso 2.

In [9]:
# Creamos la broadcast variable 
ten = 10
br_ten = sc.broadcast(ten)

# Usamos la broadcast variable
ventas_sin_impuestos = importes_int.map(lambda x: x - br_ten.value)
print(ventas_sin_impuestos.take(10))

# Destruímos la variable broadcast
br_ten.destroy()

[517.0, 376.0, 691.0, 230.0, 931.0, 17.0, 386.0, 46.0, 446.0, 138.0]


Persista el RDD ventas_sin_impuestos en los siguientes niveles de persistencia.

    1.-Memoria.
    2.-Disco solamente
    3.-Memoria y disco.

In [10]:
# 1
ventas_sin_impuestos.cache()
ventas_sin_impuestos.unpersist()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [11]:
# 2
ventas_sin_impuestos.persist(StorageLevel.DISK_ONLY)
ventas_sin_impuestos.unpersist()

PythonRDD[7] at RDD at PythonRDD.scala:53

In [12]:
# 3
ventas_sin_impuestos.persist(StorageLevel.MEMORY_AND_DISK)
ventas_sin_impuestos.unpersist()

PythonRDD[7] at RDD at PythonRDD.scala:53